# 라이브러리

In [1]:
from transformers import ElectraModel, ElectraTokenizer
from transformers import ElectraForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSemanticSegmentation, TrainingArguments, Trainer

import torch

import pandas as pd
from konlpy.tag import Mecab
import re
from tqdm import tqdm, tqdm_notebook

from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report

/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1차 모델

In [3]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./_data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
scale_data = survey[['content_id', 'scale']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
scale_data['content_id'] = scale_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
scale_data = scale_data[scale_data['content_id'] != 0].reset_index().drop(columns=['index'])
scale_data['content_id'] = scale_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./_data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(scale_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
scale_data = pd.merge(scale_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
scale_data['scale'] = scale_data['scale'].astype(int)

# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

scale_data['content'] = scale_data['content'].apply(cleaned_content)

final_df = scale_data[['content', 'scale']]

# 라벨별 개수 확인
print(final_df['scale'].value_counts())
print('\n')
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['scale'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['scale'].value_counts())
print('\n')
# 중복 데이터 제거(데이터 분리 후 중복이 생길 수 있어서 데이터 분리 후 중복 데이터 처리 진행)

# 데이터셋 개수 확인
print('중복 제거 전 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋: {}'.format(len(test_data)))
print('\n')
# 중복 데이터 제거
train_data.drop_duplicates(subset=['content'], inplace=True)
test_data.drop_duplicates(subset=['content'], inplace=True)
print('\n')
# 데이터셋 개수 확인
print('중복 제거 후 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋: {}'.format(len(test_data)))

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_63397/949624988.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scale_data['content_id'] = scale_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)


scale
 0    4675
 1     200
-1     148
Name: count, dtype: int64


train_data scale
 0    3739
 1     159
-1     120
Name: count, dtype: int64

 test_data scale
 0    940
-1     36
 1     29
Name: count, dtype: int64


중복 제거 전 학습 데이터셋: 4018
중복 제거 전 테스트 데이터셋: 1005




중복 제거 후 학습 데이터셋: 4003
중복 제거 후 테스트 데이터셋: 1001


In [ ]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가 
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))

In [ ]:
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["scale"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['scale'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './scale_model/check_point/try_1',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 10,              # 훈련 epoch 수
    per_device_train_batch_size = 16,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 64,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
# train 진행
trainer.train() 

In [ ]:
trainer.save_model("./scale_model/try_1/scale_model1")
tokenizer.save_pretrained("./scale_model/try_1/scale_tokenizer1")

In [4]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./scale_model/try_1/scale_model1"
tokenizer_path = "./scale_model/try_1/scale_tokenizer1"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data
    

100%|██████████| 1001/1001 [00:11<00:00, 85.37it/s]


,content,scale,pred
0,#1415_20221128_3인. 흐음.잘 열어보지 않으면,0,0
1,난.바보야.,0,0
2,들어가자마자 커피향이 씨게 낫다 좋앗다 하지만 인테리어는 그냥그랫다. 풀정도 되는듯.,0,0
3,3인. 스토리가 잘 감이 안 잡힌다!,0,0
4,1.5/3인. 활동성 있음.추락조심,0,0
...,...,...,...
999,나에겐 너무 어려웠던. 방탈짬바 있는 분들에게 추천,0,0
1000,2021. 12. 26. (3인) 각자 1인분씩 하고 뿌듯했던 테마.,0,0
1001,첫방 디버프로 초반에 절어버림. 볼것도 못보고 힌트썼는데 잘꾸며놓았다 . 살짝 장치...,0,0
1002,#5 - 2인 - 볼륨에 압도됨 - 히터도 방마다 빵빵함 - 다른 테마도 궁금해짐,1,1


In [6]:
print(final_df['scale'].value_counts())

scale
 0    4675
 1     200
-1     148
Name: count, dtype: int64


In [7]:
print(train_data['scale'].value_counts())
print(test_data['scale'].value_counts())

scale
 0    3724
 1     159
-1     120
Name: count, dtype: int64
scale
 0    936
-1     36
 1     29
Name: count, dtype: int64


In [9]:
print(classification_report(test_data['scale'], test_data['pred']))
test_data2 = test_data[test_data['scale'] != 0]
print(classification_report(test_data2['scale'], test_data2['pred']))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        36
           0       1.00      1.00      1.00       936
           1       0.42      0.97      0.59        29

    accuracy                           0.96      1001
   macro avg       0.47      0.65      0.53      1001
weighted avg       0.95      0.96      0.95      1001

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        36
           0       0.00      0.00      0.00         0
           1       0.44      0.97      0.61        29

    accuracy                           0.43        65
   macro avg       0.15      0.32      0.20        65
weighted avg       0.20      0.43      0.27        65



/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [10]:
test = [
    '테마 규모 대박',
    '테마 규모 개좋음',
    '규모 대박',
    '테마 규모 개쩐다.',
    '테마 규모 좁읍',
    '테마 규모 작음',
    '규모 별로임',
    '규모 콩알만함',
]

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent}, 평가: {pred}')
    print("-"*10)

문장: 테마 규모 대박, 평가: 2
----------
문장: 테마 규모 개좋음, 평가: 2
----------
문장: 규모 대박, 평가: 2
----------
문장: 테마 규모 개쩐다., 평가: 2
----------
문장: 테마 규모 좁읍, 평가: 2
----------
문장: 테마 규모 작음, 평가: 2
----------
문장: 규모 별로임, 평가: 2
----------
문장: 규모 콩알만함, 평가: 2
----------


# 2차 모델

In [4]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./_data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
scale_data = survey[['content_id', 'scale']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
scale_data['content_id'] = scale_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
scale_data = scale_data[scale_data['content_id'] != 0].reset_index().drop(columns=['index'])
scale_data['content_id'] = scale_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./_data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(scale_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
scale_data = pd.merge(scale_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
scale_data['scale'] = scale_data['scale'].astype(int)

# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

scale_data['content'] = scale_data['content'].apply(cleaned_content)

final_df = scale_data[['content', 'scale']]

# 라벨별 개수 확인
print(final_df['scale'].value_counts())
print('\n')
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['scale'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['scale'].value_counts())
print('\n')
# 중복 데이터 제거(데이터 분리 후 중복이 생길 수 있어서 데이터 분리 후 중복 데이터 처리 진행)

# 데이터셋 개수 확인
print('중복 제거 전 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋: {}'.format(len(test_data)))
print('\n')
# 중복 데이터 제거
train_data.drop_duplicates(subset=['content'], inplace=True)
test_data.drop_duplicates(subset=['content'], inplace=True)
print('\n')
# 데이터셋 개수 확인
print('중복 제거 후 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋: {}'.format(len(test_data)))

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_65645/949624988.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scale_data['content_id'] = scale_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)


scale
 0    4675
 1     200
-1     148
Name: count, dtype: int64


train_data scale
 0    3739
 1     159
-1     120
Name: count, dtype: int64

 test_data scale
 0    940
-1     36
 1     29
Name: count, dtype: int64


중복 제거 전 학습 데이터셋: 4018
중복 제거 전 테스트 데이터셋: 1005




중복 제거 후 학습 데이터셋: 4003
중복 제거 후 테스트 데이터셋: 1001


In [3]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가 
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(35080, 128)

In [5]:
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["scale"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['scale'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './scale_model/check_point/try_2',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 15,              # 훈련 epoch 수
    per_device_train_batch_size = 8,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 16,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [6]:
# train 진행
trainer.train() 

  0%|          | 0/7515 [00:00<?, ?it/s]/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_63772/1989731884.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  7%|▋         | 500/7515 [01:25<19:29,  6.00it/s]

{'loss': 0.2515, 'learning_rate': 4.6673320026613444e-05, 'epoch': 1.0}


  7%|▋         | 501/7515 [01:26<40:00,  2.92it/s]/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_63772/1989731884.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 13%|█▎        | 1000/7515 [02:51<18:04,  6.00it/s]

{'loss': 0.1418, 'learning_rate': 4.3346640053226885e-05, 'epoch': 2.0}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_63772/1989731884.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 20%|█▉        | 1500/7515 [04:16<17:22,  5.77it/s]

...

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_63772/1989731884.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|█████████▉| 7500/7515 [26:43<00:03,  4.37it/s]

{'loss': 0.0498, 'learning_rate': 9.98003992015968e-08, 'epoch': 14.97}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_63772/1989731884.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 7515/7515 [26:46<00:00,  4.68it/s]

{'train_runtime': 1606.9956, 'train_samples_per_second': 37.365, 'train_steps_per_second': 4.676, 'train_loss': 0.08479672674330091, 'epoch': 15.0}


TrainOutput(global_step=7515, training_loss=0.08479672674330091, metrics={'train_runtime': 1606.9956, 'train_samples_per_second': 37.365, 'train_steps_per_second': 4.676, 'train_loss': 0.08479672674330091, 'epoch': 15.0})

In [7]:
trainer.save_model("./scale_model/try_2/scale_model2")
tokenizer.save_pretrained("./scale_model/try_2/scale_tokenizer2")

('./scale_model/try_2/scale_tokenizer2/tokenizer_config.json',
 './scale_model/try_2/scale_tokenizer2/special_tokens_map.json',
 './scale_model/try_2/scale_tokenizer2/vocab.txt',
 './scale_model/try_2/scale_tokenizer2/added_tokens.json')

In [2]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./scale_model/try_2/scale_model2"
tokenizer_path = "./scale_model/try_2/scale_tokenizer2"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data
    

100%|██████████| 1001/1001 [00:13<00:00, 72.83it/s]


,content,scale,pred
0,#1415_20221128_3인. 흐음.잘 열어보지 않으면,0,0
1,난.바보야.,0,0
2,들어가자마자 커피향이 씨게 낫다 좋앗다 하지만 인테리어는 그냥그랫다. 풀정도 되는듯.,0,0
3,3인. 스토리가 잘 감이 안 잡힌다!,0,0
4,1.5/3인. 활동성 있음.추락조심,0,0
...,...,...,...
999,나에겐 너무 어려웠던. 방탈짬바 있는 분들에게 추천,0,0
1000,2021. 12. 26. (3인) 각자 1인분씩 하고 뿌듯했던 테마.,0,0
1001,첫방 디버프로 초반에 절어버림. 볼것도 못보고 힌트썼는데 잘꾸며놓았다 . 살짝 장치...,0,0
1002,#5 - 2인 - 볼륨에 압도됨 - 히터도 방마다 빵빵함 - 다른 테마도 궁금해짐,1,1


In [6]:
print('all_', final_df['scale'].value_counts())

print('-' * 30)

print('train_', train_data['scale'].value_counts())
print('test_', test_data['scale'].value_counts())

all_ scale
 0    4675
 1     200
-1     148
Name: count, dtype: int64
------------------------------
train_ scale
 0    3724
 1     159
-1     120
Name: count, dtype: int64
test_ scale
 0    936
-1     36
 1     29
Name: count, dtype: int64


In [7]:
print(classification_report(test_data['scale'], test_data['pred']))
test_data2 = test_data[test_data['scale'] != 0]
print(classification_report(test_data2['scale'], test_data2['pred']))

              precision    recall  f1-score   support

          -1       0.81      0.61      0.70        36
           0       1.00      1.00      1.00       936
           1       0.64      0.86      0.74        29

    accuracy                           0.98      1001
   macro avg       0.82      0.82      0.81      1001
weighted avg       0.98      0.98      0.98      1001

              precision    recall  f1-score   support

          -1       0.92      0.61      0.73        36
           0       0.00      0.00      0.00         0
           1       0.66      0.86      0.75        29

    accuracy                           0.72        65
   macro avg       0.52      0.49      0.49        65
weighted avg       0.80      0.72      0.74        65



/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [9]:
test = [
    '테마 규모 대박',
    '테마 규모 개좋음',
    '규모 대박',
    '테마 규모 개쩐다.',
    '테마 규모 좁읍',
    '테마 규모 작음',
    '규모 별로임',
    '규모 최악',
    '규모 콩알만함',
]

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent}, 평가: {pred}')
    print("-"*10)

문장: 테마 규모 대박, 평가: 2
----------
문장: 테마 규모 개좋음, 평가: 2
----------
문장: 규모 대박, 평가: 2
----------
문장: 테마 규모 개쩐다., 평가: 2
----------
문장: 테마 규모 좁읍, 평가: 2
----------
문장: 테마 규모 작음, 평가: 2
----------
문장: 규모 별로임, 평가: 2
----------
문장: 규모 최악, 평가: 2
----------
문장: 규모 콩알만함, 평가: 2
----------


In [ ]:
test = [
    '테마 규모 대박',
    '테마 규모 개좋음',
    '규모 대박',
    '테마 규모 개쩐다.',
    '테마 규모 좁읍',
    '테마 규모 작음',
    '규모 별로임',
    '규모 최악',
    '규모 콩알만함',
]

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent}, 평가: {pred}')
    print("-"*10)

문장: 테마 규모 대박, 평가: 2
----------
문장: 테마 규모 개좋음, 평가: 2
----------
문장: 규모 대박, 평가: 2
----------
문장: 테마 규모 개쩐다., 평가: 2
----------
문장: 테마 규모 좁읍, 평가: 2
----------
문장: 테마 규모 작음, 평가: 2
----------
문장: 규모 별로임, 평가: 2
----------
문장: 규모 최악, 평가: 2
----------
문장: 규모 콩알만함, 평가: 2
----------


# 3차 모델

In [2]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./_data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
scale_data = survey[['content_id', 'scale']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
scale_data['content_id'] = scale_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
scale_data = scale_data[scale_data['content_id'] != 0].reset_index().drop(columns=['index'])
scale_data['content_id'] = scale_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./_data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(scale_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
scale_data = pd.merge(scale_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
scale_data['scale'] = scale_data['scale'].astype(int)

# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

scale_data['content'] = scale_data['content'].apply(cleaned_content)

final_df = scale_data[['content', 'scale']]

# 라벨별 개수 확인
print(final_df['scale'].value_counts())
print('\n')
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['scale'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['scale'].value_counts())
print('\n')
# 중복 데이터 제거(데이터 분리 후 중복이 생길 수 있어서 데이터 분리 후 중복 데이터 처리 진행)

# 데이터셋 개수 확인
print('중복 제거 전 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋: {}'.format(len(test_data)))
print('\n')
# 중복 데이터 제거
train_data.drop_duplicates(subset=['content'], inplace=True)
test_data.drop_duplicates(subset=['content'], inplace=True)
print('\n')
# 데이터셋 개수 확인
print('중복 제거 후 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋: {}'.format(len(test_data)))

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_68418/949624988.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scale_data['content_id'] = scale_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)


scale
 0    4675
 1     200
-1     148
Name: count, dtype: int64


train_data scale
 0    3739
 1     159
-1     120
Name: count, dtype: int64

 test_data scale
 0    940
-1     36
 1     29
Name: count, dtype: int64


중복 제거 전 학습 데이터셋: 4018
중복 제거 전 테스트 데이터셋: 1005




중복 제거 후 학습 데이터셋: 4003
중복 제거 후 테스트 데이터셋: 1001


In [3]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가 
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(35080, 128)

In [4]:
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["scale"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['scale'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './scale_model/check_point/try_2',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 20,              # 훈련 epoch 수
    per_device_train_batch_size = 8,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 16,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [6]:
# train 진행
trainer.train() 

  1%|          | 68/10020 [00:22<55:48,  2.97it/s]
/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_65828/3418354174.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

                                                

{'loss': 0.2065, 'learning_rate': 4.750499001996008e-05, 'epoch': 1.0}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_65828/3418354174.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

                                                 

{'loss': 0.1209, 'learning_rate': 4.5009980039920164e-05, 'epoch': 2.0}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_65828/3418354174.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

                                                 

...

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_65828/3418354174.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

                                                  

{'loss': 0.0092, 'learning_rate': 9.98003992015968e-08, 'epoch': 19.96}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_65828/3418354174.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

100%|██████████| 10020/10020 [38:20<00:00,  4.36it/s]

{'train_runtime': 2300.2294, 'train_samples_per_second': 34.805, 'train_steps_per_second': 4.356, 'train_loss': 0.05489099013494577, 'epoch': 20.0}


TrainOutput(global_step=10020, training_loss=0.05489099013494577, metrics={'train_runtime': 2300.2294, 'train_samples_per_second': 34.805, 'train_steps_per_second': 4.356, 'train_loss': 0.05489099013494577, 'epoch': 20.0})

In [7]:
trainer.save_model("./scale_model/try_3/scale_model3")
tokenizer.save_pretrained("./scale_model/try_3/scale_tokenizer3")

('./scale_model/try_3/scale_tokenizer3/tokenizer_config.json',
 './scale_model/try_3/scale_tokenizer3/special_tokens_map.json',
 './scale_model/try_3/scale_tokenizer3/vocab.txt',
 './scale_model/try_3/scale_tokenizer3/added_tokens.json')

In [3]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./scale_model/try_3/scale_model3"
tokenizer_path = "./scale_model/try_3/scale_tokenizer3"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data
    

100%|██████████| 1001/1001 [00:13<00:00, 72.97it/s]


,content,scale,pred
0,#1415_20221128_3인. 흐음.잘 열어보지 않으면,0,0
1,난.바보야.,0,0
2,들어가자마자 커피향이 씨게 낫다 좋앗다 하지만 인테리어는 그냥그랫다. 풀정도 되는듯.,0,0
3,3인. 스토리가 잘 감이 안 잡힌다!,0,0
4,1.5/3인. 활동성 있음.추락조심,0,0
...,...,...,...
999,나에겐 너무 어려웠던. 방탈짬바 있는 분들에게 추천,0,0
1000,2021. 12. 26. (3인) 각자 1인분씩 하고 뿌듯했던 테마.,0,0
1001,첫방 디버프로 초반에 절어버림. 볼것도 못보고 힌트썼는데 잘꾸며놓았다 . 살짝 장치...,0,0
1002,#5 - 2인 - 볼륨에 압도됨 - 히터도 방마다 빵빵함 - 다른 테마도 궁금해짐,1,1


In [5]:
print('all_', final_df['scale'].value_counts())

print('-' * 30)

print('train_', train_data['scale'].value_counts())
print('test_', test_data['scale'].value_counts())

all_ scale
 0    4675
 1     200
-1     148
Name: count, dtype: int64
------------------------------
train_ scale
 0    3724
 1     159
-1     120
Name: count, dtype: int64
test_ scale
 0    936
-1     36
 1     29
Name: count, dtype: int64


In [6]:
print(classification_report(test_data['scale'], test_data['pred']))
test_data2 = test_data[test_data['scale'] != 0]
print(classification_report(test_data2['scale'], test_data2['pred']))

              precision    recall  f1-score   support

          -1       0.97      1.00      0.99        36
           0       1.00      1.00      1.00       936
           1       0.90      0.97      0.93        29

    accuracy                           1.00      1001
   macro avg       0.96      0.99      0.97      1001
weighted avg       1.00      1.00      1.00      1001

              precision    recall  f1-score   support

          -1       0.97      1.00      0.99        36
           1       1.00      0.97      0.98        29

    accuracy                           0.98        65
   macro avg       0.99      0.98      0.98        65
weighted avg       0.99      0.98      0.98        65



In [7]:
test = [
    '테마 규모 대박',
    '테마 규모 개좋음',
    '규모 대박',
    '테마 규모 개쩐다.',
    '테마 규모 좁읍',
    '테마 규모 작음',
    '규모 별로임',
    '규모 최악',
    '규모 콩알만함',
]

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent}, 평가: {pred}')
    print("-"*10)

문장: 테마 규모 대박, 평가: 2
----------
문장: 테마 규모 개좋음, 평가: 2
----------
문장: 규모 대박, 평가: 2
----------
문장: 테마 규모 개쩐다., 평가: 2
----------
문장: 테마 규모 좁읍, 평가: 0
----------
문장: 테마 규모 작음, 평가: 0
----------
문장: 규모 별로임, 평가: 2
----------
문장: 규모 최악, 평가: 2
----------
문장: 규모 콩알만함, 평가: 2
----------


# 4차 모델

In [4]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./_data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
scale_data = survey[['content_id', 'scale']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
scale_data['content_id'] = scale_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
scale_data = scale_data[scale_data['content_id'] != 0].reset_index().drop(columns=['index'])
scale_data['content_id'] = scale_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./_data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(scale_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
scale_data = pd.merge(scale_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
scale_data['scale'] = scale_data['scale'].astype(int)
final_df = scale_data[['content', 'scale']]

# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

from kiwipiepy import Kiwi
kiwi = Kiwi()

def kiwi_clean(text):
    get_kiwi_pos = ['NNG', 'NP', 'NNP', 'MM', 'VV', 'VV-I', 'VV-R', 'VA', 'VA-I', 'VA-R', 'VCP', 'VCN', 'MAG', 'MAJ', 'XR']
    kiwi_lem = []
    for word in kiwi.tokenize(text):
        if word.tag in get_kiwi_pos:
            kiwi_lem.append(word.lemma)
    return ' '.join(kiwi_lem)

final_df['content'] = final_df['content'].apply(cleaned_content)
final_df['content'] = final_df['content'].apply(kiwi_clean)

final_df = final_df.drop_duplicates()
final_df = final_df.reset_index().drop(columns = ['index'])

# 라벨별 개수 확인
print(final_df['scale'].value_counts())
print('\n')
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['scale'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['scale'].value_counts())

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_93317/3482852253.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scale_data['content_id'] = scale_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)
/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_93317/3482852253.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['content'] = final_df['content'].apply(cleaned_content)


scale
 0    4545
 1     200
-1     147
Name: count, dtype: int64


train_data scale
 0    3639
 1     159
-1     116
Name: count, dtype: int64

 test_data scale
 0    913
-1     36
 1     29
Name: count, dtype: int64


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_93317/3482852253.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['content'] = final_df['content'].apply(kiwi_clean)


In [6]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가 
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(35080, 128)

In [7]:
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["scale"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['scale'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './scale_model/check_point/try_4',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 30,              # 훈련 epoch 수
    per_device_train_batch_size = 8,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 16,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [8]:
# train 진행
trainer.train() 

  0%|          | 0/14700 [00:00<?, ?it/s]/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_74840/3346513247.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  3%|▎         | 500/14700 [01:24<39:36,  5.98it/s]  

{'loss': 0.2566, 'learning_rate': 4.8299319727891155e-05, 'epoch': 1.02}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_74840/3346513247.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  7%|▋         | 1000/14700 [02:50<39:24,  5.79it/s] 

{'loss': 0.1874, 'learning_rate': 4.6598639455782315e-05, 'epoch': 2.04}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_74840/3346513247.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 10%|█         | 1500/14700 [04:16<37:50,  5.81it/s]  

...

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_74840/3346513247.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 99%|█████████▊| 14500/14700 [40:30<00:33,  6.01it/s]

{'loss': 0.0397, 'learning_rate': 6.802721088435375e-07, 'epoch': 29.59}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_74840/3346513247.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 14700/14700 [41:04<00:00,  5.97it/s]

{'train_runtime': 2464.3528, 'train_samples_per_second': 47.647, 'train_steps_per_second': 5.965, 'train_loss': 0.08986154724951503, 'epoch': 30.0}


TrainOutput(global_step=14700, training_loss=0.08986154724951503, metrics={'train_runtime': 2464.3528, 'train_samples_per_second': 47.647, 'train_steps_per_second': 5.965, 'train_loss': 0.08986154724951503, 'epoch': 30.0})

In [9]:
trainer.save_model("./scale_model/try_4/scale_model4")
tokenizer.save_pretrained("./scale_model/try_4/scale_tokenizer4")

('./scale_model/try_4/scale_tokenizer4/tokenizer_config.json',
 './scale_model/try_4/scale_tokenizer4/special_tokens_map.json',
 './scale_model/try_4/scale_tokenizer4/vocab.txt',
 './scale_model/try_4/scale_tokenizer4/added_tokens.json')

In [2]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./scale_model/try_4/scale_model4"
tokenizer_path = "./scale_model/try_4/scale_tokenizer4"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data
    

  0%|          | 0/978 [00:00<?, ?it/s]

100%|██████████| 978/978 [00:10<00:00, 96.48it/s] 


,content,scale,pred
0,인 활동 있다 추락 조심,0,0
1,인 나 눈 없다 치다,0,0
2,인테리어 향기 굿 문제 전반 쉽다 문제 이다 같다 나 능지 얼마나 낮다 알다 있다,0,0
3,문제 재밌다 타임 어택 느낌 좋다 급하다 하다 스토리 못 즐기다 아쉽다,0,0
4,첫 탈출 우왕좌왕 그래도 거의 탈출 코앞 이다,0,0
...,...,...,...
973,나 너무 어렵다 방탈짬바 있다 추천,0,0
974,인 각자 하다 뿌듯하다 테마,0,0
975,첫 디버프로 초반 절다 보다 못 보다 힌트 쓰다 잘 꾸미다 살짝 장치 오류 아쉽다,0,0
976,인 볼륨 압도 히터 방 빵빵하다 다른 테마 궁금,1,-1


In [6]:
print('0포함', '\n', classification_report(test_data['scale'], test_data['pred']))
test_data2 = test_data[test_data['scale'] != 0]
print('0 제외', '\n', classification_report(test_data2['scale'], test_data2['pred']))

0포함 
               precision    recall  f1-score   support

          -1       0.90      0.72      0.80        36
           0       1.00      0.99      1.00       913
           1       0.70      0.90      0.79        29

    accuracy                           0.98       978
   macro avg       0.86      0.87      0.86       978
weighted avg       0.98      0.98      0.98       978

0 제외 
               precision    recall  f1-score   support

          -1       0.96      0.72      0.83        36
           0       0.00      0.00      0.00         0
           1       0.76      0.90      0.83        29

    accuracy                           0.80        65
   macro avg       0.58      0.54      0.55        65
weighted avg       0.87      0.80      0.83        65



/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [8]:
test = [
    '테마 규모 대박',
    '테마 규모 개좋음',
    '규모 대박',
    '테마 규모 개쩐다.',
    '테마 규모 좁읍',
    '테마 규모 작음',
    '규모 별로임',
    '규모 최악',
    '규모 콩알만함',
    '스토리 쓰레기'
]

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent}, 평가: {pred}')
    print("-"*10)

문장: 테마 규모 대박, 평가: 2
----------
문장: 테마 규모 개좋음, 평가: 2
----------
문장: 규모 대박, 평가: 0
----------
문장: 테마 규모 개쩐다., 평가: 2
----------
문장: 테마 규모 좁읍, 평가: 0
----------
문장: 테마 규모 작음, 평가: 0
----------
문장: 규모 별로임, 평가: 0
----------
문장: 규모 최악, 평가: 2
----------
문장: 규모 콩알만함, 평가: 0
----------
문장: 스토리 쓰레기, 평가: 1
----------
